# Deployment Track Report

* Name: Ziyi Liang
* Net-ID: zl5604

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil
from PIL import Image

import tensorflow.keras.backend as K
import tensorflow as tf

## Models

### Previous Model

### Submit Model (MobileNet)

* I used the built-in MobileNet. The model has 3,228,864 paramemters.
* The augmentations used are the default ones provided by Professor Fund. (Explanation for these augmentations quoted from docstring of ` tf.keras.preprocessing.image.ImageDataGenerator`)
    * `rescale=1.0/255`: Rescale all pixels into [0, 1] range;
    * `rotation_range=1`: degree range for random rotations;
	* `zoom_range=0.1`：range for random zoom;
	* `width_shift_range=0.1`：fraction of total width shifted horizontally;
	* `height_shift_range=0.1`：fraction of total height shifted vertically;
	* `shear_range=0.1`：the intensity of image being sheared;
	* `horizontal_flip=True`：image is filpped horizontally randomly;
	* `fill_mode="nearest"`: points outside the boundaries of the input are filled with nearest mode;
* I trained the classification head 35 epochs, with Adam optimizer, learning rate of 0.001, amd batch size of 32. The validation accuracy at the end of this training stage is 0.8318.
* There are three stages of un-freezing:
	1. Last 7 layers un-frozen: 
		- epoch: 28, optimizer: Adam, learning rate: 1e-05, batch size: 32, validation accuracy: 0.8370;
	2. Last 14 layers un-frozen:
		- epoch: 30, optimizer: Adam, learning rate: 5e-06, batch size: 32, validation accuracy: 0.8516;
	3. Last 20 layers un-frozen:
		- epoch: 8, optimizer: Adam, learning rate: 2.5e-06, batch size: 32, validation accuracy: 0.8513;
	Since the third stage of un-freezing has no improvement in accucary, the final model is the second stage model (14 layers un-frozen)
* The final accuracy is 0.8742
* My model correctly predict the calss of my custom test image.


### Submit Model (VGG16)

### Model Summary

## Deployments

## Evaluation